In [ ]:
import torch
import torch.nn as nn

In [ ]:
torch.__version__

In [ ]:
from torch.utils.data import Dataset as TorchDataset
from torch.utils.data import DataLoader

In [ ]:
import numpy as np
import datetime
import os, sys
import math
import matplotlib.pyplot as plt
dir_path = os.path.dirname(os.getcwd() + "/../src/")
sys.path.insert(1, dir_path)
import data_manip

np.set_printoptions(precision=4)

In [ ]:
import pandas as pd
import os
pd.set_option("display.max_columns", None)

In [ ]:
MODEL_NAME = 'DA'
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(DEVICE)

In [ ]:
def _reformatForMLADDA(df: pd.DataFrame, drop_fields: list, unique_fields: list, label_field: str, benign_label:str,  target, b_resample, balance=False):
    """
    Reformat DataFrame for Machine Learning algorithms
    """
    df = df.drop(drop_fields, axis=1)
    df = df.replace(np.inf, np.nan)
    df = df.dropna()
    if b_resample != None:
        ben_df = df[df[label_field] == benign_label]
        ben_df = ben_df.sample(frac=b_resample)
        df = df.drop(df[df[label_field] == benign_label].index)
        df = pd.concat([ben_df, df])
    df = df.drop_duplicates()
    if benign_label == None:
        df[label_field].loc[(df[label_field] != target)] = 'Other'
        label_mapping = {"Other": 0,
                        target: 1}
    elif type(target) == list:
        label_index = df[label_field].loc[((df[label_field] != benign_label) & (~df[label_field].isin(target)))].index
        df = df.drop(label_index)
        unique_labels = df[label_field].unique()
        print(unique_labels)
        ones = [1 for x in range(len(unique_labels))]
        label_mapping = dict(zip(unique_labels, ones))
        label_mapping[benign_label] = 0
    else:
        label_index = df[label_field].loc[((df[label_field] != benign_label) & (df[label_field] != target))].index
        df = df.drop(label_index)
        label_mapping = {benign_label: 0,
                        target: 1}
    if balance:
        g = df.groupby(label_field)
        df = g.apply(lambda x: x.sample(g.size().min()).reset_index(drop=True))
    df[label_field] = df[label_field].replace(label_mapping)
    labels = df[label_field]
    df = df.drop(label_field, axis=1)
    if unique_fields is not None:
        for field in unique_fields:
            unique_field = df[field].unique()
            field_mapping = dict(zip(unique_field, range(len(unique_field))))
            df = df.replace({field: field_mapping})

    return df, labels


def reformatForMLADDA(df: pd.DataFrame, metadata: dict, target: str, b_resample: int, balance=False):
    """
    Reformat DataFrame for Machine Learning algorithms
    """
    df, labels = _reformatForMLADDA(df, metadata["drop_fields"], metadata["unique_fields"], metadata["label_field"], metadata['benign_label'], target, b_resample, balance)
    return df, labels

def reformatForADDA(df, metadata, target, b_resample1, b_resample2, target_limit, target_test_limit, remove=False):
    """
    Reformat data in manner outlined by ADDA, resampling the benign/target traffic where necessary.
    """
    classes = df[metadata["label_field"]].value_counts().index.tolist()
    half = int(df.shape[0] / 2)
    classes.remove(metadata["benign_label"])
    classes.remove(target)
    
    print("Classes: ", classes)
    
    df1 = df.iloc[:half]
    df2 = df.iloc[half:]
    del(df)
    source, source_labels = reformatForMLADDA(df1, metadata, classes, b_resample1)
    if remove:        
        # A very large number of attack traffic has the below properties across multiple classes. We remove it to reduce overlap between classes.
        # We justify this in more detail below.
        source = source.drop(source[(((source["spkts"] == 10) & (source["dpkts"] == 6)) | 
                                                                                  ((source["spkts"] == 10) & (source["dpkts"] == 8)) |
                                                                                 ((source["spkts"] == 2) & (source["dpkts"] == 0)))].index)
    source_labels = source_labels.loc[source.index]
    print("Finished Processing Source")
    full_target, full_target_labels = reformatForMLADDA(df2, metadata, target, b_resample2, balance=True)

    if remove:
        # Same as above
        trimmed_target = full_target.drop(full_target[(((full_target["spkts"] == 10) & (full_target["dpkts"] == 6)) | 
                                                                                      ((full_target["spkts"] == 10) & (full_target["dpkts"] == 8)) |
                                                                                        ((full_target["spkts"] == 2) & (full_target["dpkts"] == 0)))].index)
        print(trimmed_target.shape)

        trimmed_target = trimmed_target.groupby(metadata["label_field"])
        trimmed_target = pd.DataFrame(trimmed_target.apply(lambda x: x.sample(trimmed_target.size().min()).reset_index(drop=True)))
        trimmed_target_labels = full_target_labels.loc[trimmed_target.index] # replace with trimmed
        target = trimmed_target.sample(n=target_limit, random_state=100)
        target_labels = trimmed_target_labels.loc[target.index]
        target_test, target_test_labels = trimmed_target.drop(target.index), trimmed_target_labels.drop(target_labels.index)
        target_test = target_test.sample(n=target_test_limit, random_state=100)
        target_test_labels = target_test_labels.loc[target_test.index]
    else:
        target = full_target.sample(n=target_limit, random_state=100)
        target_labels = full_target_labels.loc[target.index]
        target_test, target_test_labels = full_target.drop(target.index), full_target_labels.drop(target_labels.index)
        target_test = target_test.sample(n=target_test_limit, random_state=100)
        target_test_labels = target_test_labels.loc[target_test.index]
    return source, source_labels, target, target_labels, target_test, target_test_labels


In [ ]:
class Dataset(TorchDataset):
    """Characterizes a dataset for PyTorch"""
    def __init__(self, data, labels):
        """Initialization"""
        self.labels = labels
        self.data = data

    def __len__(self):
        """Denotes the total number of samples"""
        return self.data.shape[0]

    def __getitem__(self, index):
        """Generates one sample of data"""
        X = self.data[index]
        y = self.labels[index]

        return X, y

In [ ]:
class Base(nn.Module):
    """
        Base Model to compare with
    """
    def __init__(self, input_size=26, num_classes=2):
        super(Base, self).__init__()
        
        self.lin1 = nn.Linear(input_size, 64)
        self.bn1 = nn.BatchNorm1d(64)
        self.r1 = nn.ReLU(inplace=True)
        self.lin2 = nn.Linear(64, 32)
        self.bn2 = nn.BatchNorm1d(32)
        self.r2 = nn.ReLU(inplace=True)
        self.lin3 = nn.Linear(32, 16)
        self.bn3 = nn.BatchNorm1d(16)
        self.r3 = nn.ReLU(inplace=True)
        self.lin4 = nn.Linear(16, 2)
        self.soft = nn.LogSoftmax()
        
        
    def forward(self, x):
        x = self.lin1(x)
        x = self.bn1(x)
        x = self.r1(x)
        x = self.lin2(x)
        x = self.bn2(x)
        x = self.r2(x)
        x = self.lin3(x)
        x = self.bn3(x)
        x = self.r3(x)
        x = self.lin4(x)
        pred = self.soft(x)
        
        return pred

In [ ]:
class Generator(nn.Module):
    """
        Generator/Classifier
    """
    def __init__(self, input_size=26, num_classes=2):
        super(Generator, self).__init__()

        self.lin1 = nn.Linear(input_size, 64)
        self.bn1 = nn.BatchNorm1d(64)
        self.r1 = nn.ReLU(inplace=True)
        self.lin2 = nn.Linear(64, 32)
        self.bn2 = nn.BatchNorm1d(32)
        self.r2 = nn.ReLU(inplace=True)
        self.lin3 = nn.Linear(32, 16)
        self.bn3 = nn.BatchNorm1d(16)
        self.r3 = nn.ReLU(inplace=True)
        self.lin4 = nn.Linear(16, 2)
        self.soft = nn.LogSoftmax()
        
        
    def forward(self, x):
        x = self.lin1(x)
        x = self.bn1(x)
        x = self.r1(x)
        x = self.lin2(x)
        x = self.bn2(x)
        x = self.r2(x)
        x = self.lin3(x)
        x = self.bn3(x)
        embed = self.r3(x)
        pred = self.lin4(embed)
        pred = self.soft(pred)
        
        return embed, pred

In [ ]:
class Discriminator(nn.Module):
    """
        Simple Discriminator w/ MLP
    """
    def __init__(self, input_size=16, num_classes=2):
        super(Discriminator, self).__init__()
        self.layer = nn.Sequential(
            nn.Linear(input_size, 64),
            nn.BatchNorm1d(64),
            nn.ReLU(inplace=True),
            nn.Linear(64, 32),
            nn.BatchNorm1d(32),
            nn.ReLU(inplace=True),
            nn.Linear(32, 16),
            nn.BatchNorm1d(16),
            nn.ReLU(inplace=True),
            nn.Linear(16, 2),
            nn.Softmax()
        )
        
    def forward(self, h):
        c = self.layer(h)
        return c

The last layer of the
generator serves as input to the discriminator as well as feeds into
a soft-max layer to predict the class of the data sample. Similarly,
the final layer of the discriminator feeds into a soft-max layer to
predict the domain the sample belongs to.

In [ ]:
def GenDomainLoss(dg_tgt):
    return -(torch.mean(torch.log(dg_tgt[:, :1])))

def DiscClassLoss(source_probs, target_probs):
    log_source_probs = torch.log(source_probs[:, :1])
    log_target_probs = torch.log(1 - target_probs[:, :1])
    left = torch.mean(log_source_probs)
    right = torch.mean(log_target_probs)
    return -(left + right)

def set_requires_grad(model, requires_grad=True):
    for param in model.parameters():
        param.requires_grad = requires_grad
        
GenClassLoss = nn.NLLLoss()
BaseLoss = nn.CrossEntropyLoss()

In [ ]:
original = True # True: Recreate original pipeline. False: Drop features/flows that are hihgly discriminitive across multiple classes.

unsw_direc = None # Put directory containing UNSW NB15 CSV files here
target_train_limit = 100
target_test_limit = 10000

if original:
    unsw_metadata_file = "../metadata/unsw/metadata.json" # Put original metadata file here i.e., /unsw/metadata.json 
else:
    unsw_metadata_file = "../metadata/unsw/metadata_adda.json" #  Put adda metadata file here i.e., /unsw/metadata_adda.json 
metadata = data_manip.readMetadata(unsw_metadata_file)
df = data_manip.readDirec(unsw_direc, metadata)

We note that, due to the 'strikes' contained within UNSW NB15, Malicious flows are clustered around a small number of flow sizes whilst Benign flows are not. In particular, a model can achieve extremely good cross-class generalisation based on the 'dpkt' and 'spkt' features alone. This occurs across completely distinct attacks where there is no reasonable justification for this generalisation. Other features (ttls, rtts, etc.) are similarly discriminative. As a result, evaluating a model's cross-class generalisation capabilities on UNSW NB15 is difficult to justify.

In [ ]:
if original:
    print(df[(((df["spkts"] == 10) & (df["dpkts"] == 6)) |
     ((df["spkts"] == 10) & (df["dpkts"] == 8)) |
     ((df["spkts"] == 2) & (df["dpkts"] == 0)))][metadata["label_field"]].value_counts() / df[metadata["label_field"]].value_counts())

In [ ]:

if original:
    remove=False
    benign_resample = 0.05 # We resample the benign traffic to approximately match the original ADDA amounts
else:
    remove=True
    benign_resample = 0.015 # We change the sample rate so the benign:malicious ratio remains approximately the same
source, source_labels, target, target_labels, test, test_labels = reformatForADDA(df, metadata, "Exploits", benign_resample, .1, target_train_limit, target_test_limit, remove=remove)
source, source_labels, target, target_labels, test, test_labels = source.to_numpy(), source_labels.to_numpy(), target.to_numpy(), target_labels.to_numpy(), test.to_numpy(), test_labels.to_numpy()
df = df.drop(metadata["drop_fields"], axis=1)
input_size = len(df.columns.tolist())
del df

In [ ]:
B = Base(input_size=input_size).double().to(DEVICE)
G = Generator(input_size=input_size).double().to(DEVICE)
D = Discriminator().double().to(DEVICE)

G_opt = torch.optim.Adam(G.parameters())
D_opt = torch.optim.Adam(D.parameters())
B_opt = torch.optim.Adam(B.parameters())

batch_size = 32

max_epoch_base = 10000
step_adda = 0
step_base = 0

ll_g, ll_d = [], []
acc_lst = []

In [ ]:
# Paper claims to train for 10000 iterations --- we assume that means over the generator training set
max_epoch_adda = math.ceil(10000 / (target_train_limit / (batch_size / 2)))
print(max_epoch_adda)

In [ ]:
source_dataset = Dataset(source, source_labels) # Define Source Dataset
target_dataset = Dataset(target, target_labels) # Define Target Dataset
test_dataset = Dataset(test, test_labels)

source_dataloader = DataLoader(source_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
target_dataloader = DataLoader(target_dataset, batch_size=batch_size, shuffle=True, num_workers=2)
test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True, num_workers=2)

In [ ]:
def adda_pipeline(step_adda):
    with torch.autograd.set_detect_anomaly(True):
        for epoch in range(1, max_epoch_adda+1):
            d_loss_avg = []
            gd_loss_avg = []
            gc_loss_avg = []
            for idx, ((src_entries, src_class_labels), (tgt_entries, tgt_class_labels)) in enumerate(zip(source_dataloader, target_dataloader)):

                ##########################
                # Training Discriminator #
                ##########################


                # Get Source, Target and Class Labels
                src, sc_labels, tgt, tc_labels = src_entries.to(DEVICE), src_class_labels.to(DEVICE), tgt_entries.to(DEVICE), tgt_class_labels.to(DEVICE)
                labels = torch.cat([sc_labels, tc_labels], dim=0)


                # Freeze Gradients of G
                set_requires_grad(G, requires_grad=False)
                set_requires_grad(D, requires_grad=True)

                # Combine source and target into single batch
                d_samples = torch.cat([src, tgt], dim=0)

                # Get Source Labels
                src_labels = torch.ones_like(src)
                tgt_labels = torch.zeros_like(tgt)
                source_labels = torch.cat([src_labels, tgt_labels])

                # Get embedding and label predictions for all samples
                src_outputs, src_label_preds = G(src.double())
                tgt_outputs, tgt_label_preds = G(tgt.double())


                # Get Source Predictions for all
                src_source_preds = D(src_outputs)
                tgt_source_preds = D(tgt_outputs)

                # Calculate the loss
                disc_loss = DiscClassLoss(src_source_preds, tgt_source_preds)

                #Zero out gradients
                D.zero_grad()

                # Backpropogate
                disc_loss.backward(retain_graph=True)

                # Update Optimiser
                D_opt.step() 


                ######################
                # Training Generator #
                ######################

                set_requires_grad(D, requires_grad=False)
                set_requires_grad(G, requires_grad=True)

                # Get Target Outputs Only
                # tgt_outputs, tgt_labels

                # Get Source Prediction of only the Targets
                outputs, label_preds = G(d_samples.double())
                new_tgt_source_preds = D(tgt_outputs)


                # Calc Domain Loss
                gen_domain_loss = GenDomainLoss(new_tgt_source_preds)

                ## Calc Classification Loss
                gen_class_loss = GenClassLoss(label_preds, labels)


                # Zero Out Gradients
                G.zero_grad()

                # Calc combined loss and backpropogate
                comb_loss = ((gen_domain_loss + gen_class_loss) / 2)
                comb_loss.backward()


                # Update Optimiser
                G_opt.step()

                if (epoch % 50 == 0) and (epoch != 0):
                    d_loss_avg.append(disc_loss.item())
                    gd_loss_avg.append(gen_domain_loss.item())
                    gc_loss_avg.append(gen_class_loss.item())

            if (epoch % 50 == 0) and (epoch != 0):
                dt = datetime.datetime.now().strftime('%H:%M:%S')
                print('Epoch: {}/{}, Step: {}, D Loss: {:.4f}, G Domain Loss: {:.4f}, G Class Loss: {:.4f} ---- {}'.format(epoch, max_epoch_adda, idx, np.mean(d_loss_avg), np.mean(gd_loss_avg), np.mean(gc_loss_avg), dt))
                ll_g.append(comb_loss)
                ll_d.append(disc_loss)

            if (epoch % 200 == 0) and (idx != 0):
                print("[!] [Src Source Preds]\n+++++(Want 0 Acc)+++++\n", torch.mean(torch.argmax(label_preds, dim = 1).double()))
                print("[!] [Target Source Preds]\n++++++(Want 1 Acc)++++++\n", torch.mean(torch.argmax(tgt_source_preds, dim=1).double()))
                print("[=====================================]")
                print("[!] Gen Acc: ", 1 - torch.mean(torch.abs(labels - torch.argmax(label_preds, dim = 1)).double()))
                print("[=====================================]")

            if (epoch % 50 == 0) and (idx != 0):
                G.eval()
                D.eval()
                corrects = torch.zeros(1).to(DEVICE)
                for idx, (tgt, labels) in enumerate(test_dataloader):
                    tgt, labels = tgt.to(DEVICE), labels.to(DEVICE)
                    outputs, label_preds = G(tgt)
                    _, preds = torch.max(label_preds, 1)
                    corrects += (preds == labels).sum()
                acc = corrects.item() / len(test_dataloader.dataset)
                print('***** Test Result: {:.4f}, Step: {}'.format(acc, step_adda))
                acc_lst.append(acc)

                G.train()
                D.train()
            step_adda += 1

In [ ]:
def base_pipeline(step_base):    
    with torch.autograd.set_detect_anomaly(True):
        for epoch in range(1, max_epoch_base+1):
            bc_loss_avg = []
            for idx, ((src_entries, src_class_labels), (tgt_entries, tgt_class_labels)) in enumerate(zip(source_dataloader, target_dataloader)):

                ##########################
                # Training Discriminator #
                ##########################


                # Get Source, Target and Class Labels
                src, sc_labels, tgt, tc_labels = src_entries.to(DEVICE), src_class_labels.to(DEVICE), tgt_entries.to(DEVICE), tgt_class_labels.to(DEVICE)
                labels = torch.cat([sc_labels, tc_labels], dim=0)

                # Combine source and target into single batch
                d_samples = torch.cat([src, tgt], dim=0)


                # Get embedding and label predictions for all samples
                label_preds = B(d_samples.double())

                # Calculate the loss
                class_loss = BaseLoss(label_preds, labels)

                # Zero Out Gradients
                B.zero_grad()

                # Send loss backwards
                class_loss.backward()

                # Update Optimiser
                B_opt.step()

                if (epoch % 50 == 0) and (epoch != 0):
                    bc_loss_avg.append(class_loss.item())

            if (epoch % 100 == 0) and (epoch != 0):
                dt = datetime.datetime.now().strftime('%H:%M:%S')
                print('Epoch: {}/{}, Step: {}, B Loss: {:.4f} ---- {}'.format(epoch, max_epoch_base, idx, np.mean(bc_loss_avg), dt))
            if (epoch % 1000 == 0) and (epoch != 0):
                print(epoch)

                print("[=====================================]")
                print("[!] Gen Acc: ", 1 - torch.mean(torch.abs(labels - torch.argmax(label_preds, dim = 1)).double()))
                print("[=====================================]")


            if (epoch % 100 == 0) and (epoch != 0):
                B.eval()
                corrects = torch.zeros(1).to(DEVICE)
                for idx, (tgt, labels) in enumerate(test_dataloader):
                    tgt, labels = tgt.to(DEVICE), labels.to(DEVICE)
                    label_preds = B(tgt)
                    _, preds = torch.max(label_preds, 1)
                    corrects += (preds == labels).sum()
                acc = corrects.item() / len(test_dataloader.dataset)
                print('***** Test Result: {:.4f}, Step: {}'.format(acc, step_base))
                acc_lst.append(acc)

                B.train()
            step_base += 1

In [ ]:
adda_pipeline(step_adda)

In [ ]:
B = Base().double().to(DEVICE)
B_opt = torch.optim.Adam(B.parameters())
base_pipeline(step_base)